In [1]:
from langchain.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.manager import CallbackManager
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain import hub
from langchain.schema.runnable import RunnablePassthrough

In [2]:
import time

In [3]:
llm = Ollama(model="Llama-3-Open-Ko-8B-Instruct-preview-Q6_K",
             callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

In [4]:
#모델 확인용
prompt = ChatPromptTemplate.from_template(
    "Answer the following question in Korean(hangul).\n\nQuestion: {question}\n\nAnswer:"
)

chain = prompt | llm | StrOutputParser()

_ = chain.invoke({"question": "안녕"})

안녕하세요! 저는 챗봇입니다. 무엇을 도와드릴까요? 문의하세요! :) <font face="맑은 고딕, 맑은색">. 󰡔

In [5]:
loader = PyPDFLoader('./소나기 - 황순원.pdf')
document = loader.load()
print(document[0].page_content[:200])

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(document)

embeddings = HuggingFaceEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)
retriever = docsearch.as_retriever()

- 1 -소나기
황순원
소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀 (曾孫女 )딸이라는 걸 알 수 있었다 . 
소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다 . 서울서는 이런 개울물을 보지 
못하기나 한 듯이.
벌써 며칠째 소녀는 , 학교에서 돌아오는 길에 물장난이었다 . 그런데 , 어제까지 개울 기슭에
서 하더니 , 오늘은 징검다리 한가운


In [6]:
custom_rag_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If you don't know the answer, just say that you don't know.
            Use three sentences maximum and keep the answer concise.
            Use Korean for question-answering:
            \n\n
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

In [7]:
# RAG Chain 구성
custom_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
)

In [9]:
start = time.time()

In [9]:
response = custom_rag_chain.invoke("'소나기' 속 등장인물의 이름은 뭐야?")
print(response)

print("time :", time.time() - start)
start = time.time()

소나기의 주인공은 소년과 소녀입니다. 소년은 개울가에서 소녀를 보게 되고, 소녀는 물장난을 하다가 사라진 후에 다시 나타나는 등 여러 가지 모험을 겪습니다. 소녀는 소년에게 도움을 주거나 방해를 하는 인물로 등장합니다. 소년은 소녀의 존재를 알게 되면서 다양한 사건과 만남을 가집니다. 소녀는 소년에게 많은 영향을 미칩니다. 소년은 소녀와의 만남으로 성장하고 변화하는 과정을 겪습니다. 소나기는 소년과 소녀의 관계를 중심으로 이야기를 전개합니다. 소나기의 주인공은 소년과 소녀입니다. 소년은 개울가에서 소녀를 보게 되고, 소녀는 물장난을 하다가 사라진 후에 다시 나타나는 등 여러 가지 모험을 겪습니다. 소녀는 소년에게 도움을 주거나 방해를 하는 인물로 등장합니다. 소년은 소녀의 존재를 알게 되면서 다양한 사건과 만남을 가집니다. 소녀는 소년에게 많은 영향을 미칩니다. 소년은 소녀와의 만남으로 성장하고 변화하는 과정을 겪습니다. 소나기는 소년과 소녀의 관계를 중심으로 이야기를 전개합니다. 
time : 449.4881052970886


In [10]:
response = custom_rag_chain.invoke("'소나기' 의 줄거리를 요약해줘")
print(response)

print("time :", time.time() - start)
start = time.time()

Here is a summary of the story. The story begins with a boy who sees a girl playing in a stream and becomes fascinated with her. He tries to approach her, but she disappears, leaving only a few clues behind. The boy then meets a mysterious old man who gives him a red stone, which he keeps in his pocket. As time passes, the boy becomes more and more curious about the girl's whereabouts. One day, the boy finds a piece of cloth with a red thread on it, and he decides to follow the stream to find her. Along the way, he encounters various creatures and experiences strange events. The story is full of symbolism and metaphors, and the ending is open-ended, leaving the reader wondering what will happen next. Would you like me to continue the summary?
time : 441.2430193424225


In [11]:
rag_prompt = hub.pull("rlm/rag-prompt")

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
)

In [12]:
start = time.time()

In [13]:
response = rag_chain.invoke("'소나기' 속 등장인물의 이름은 뭐야?")
print(response)

print("time :", time.time() - start)
start = time.time()

So, the main character in '소나기' is a young boy who sees a girl playing with a ball in a river. The girl disappears and the boy searches for her everywhere. He meets a old man who tells him to follow the stream and find her. The old man gives him a red thread, which leads him to a strange forest where he finds a mysterious woman. The woman is a shaman who helps him find his lost love. The boy then becomes a shaman himself and performs a ritual to summon the girl's spirit. After the ritual, the boy returns home and finds the girl in his dream. The story is about the boy's journey to find his lost love and become a shaman. The red thread symbolizes the connection between the two characters. The story is a metaphor for the search for one's true self and the connection with nature. The boy's journey is a journey of self-discovery and spiritual growth. The story is a beautiful and mysterious tale that explores the themes of love, identity, and spirituality.
So, the main character in '소나기' is

In [14]:
response = rag_chain.invoke("'소나기' 의 줄거리를 요약해줘")
print(response)

print("time :", time.time() - start)
start = time.time()

You are asking about the summary of '소나기' novel. The story is about a boy who sees a girl playing in a stream and becomes fascinated with her. He tries to follow her, but she disappears, and he continues to see her everywhere. One day, he finds a red clay ball in the stream and takes it, and then he meets an old man who tells him that the girl is his daughter from a past life. The boy's heart beats fast as he thinks about the girl, and he becomes more and more obsessed with her. He tries to find her everywhere, but she disappears again. One day, he finds a red clay ball in the stream again, and this time, he meets an old man who tells him that the girl is his daughter from a past life. The boy's heart beats fast as he thinks about the girl, and he becomes more and more obsessed with her. He tries to find her everywhere, but she disappears again. One day, he finds a red clay ball in the stream again, and this time, he meets an old man who tells him that the girl is his daughter from a p